In [ ]:
!apt-get install fluidsynth -y  # Instala o FluidSynth no sistema
!pip install pyfluidsynth pretty_midi numpy torch torchaudio
!pip install audio-diffusion-pytorch

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# Agora você pode acessar seus arquivos em '/content/drive/My Drive'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import numpy as np
import pretty_midi
import subprocess
import torch
import torchaudio
from torch.utils.data import Dataset, DataLoader

class NpyToWavDataset(Dataset):
    def __init__(self, npy_directory, output_directory, soundfont_path, fluidsynth_path, sample_rate=22050, duration=12):
        self.file_paths = [os.path.join(npy_directory, f) for f in os.listdir(npy_directory) if f.endswith('.npy')]
        self.output_directory = output_directory
        self.soundfont_path = soundfont_path
        self.fluidsynth_path = fluidsynth_path
        self.sample_rate = sample_rate
        self.num_samples = sample_rate * duration

        # Cria o diretório de saída, se necessário
        os.makedirs(output_directory, exist_ok=True)

    def piano_roll_to_midi(self, piano_roll, midi_file):
        pm = pretty_midi.PrettyMIDI(initial_tempo=120, resolution=480)  # Define tempo e resolução
        instrument = pretty_midi.Instrument(program=0)  # Piano

        for pitch, roll in enumerate(piano_roll):
            notes = np.where(roll > 0)[0]
            for note in notes:
                start = note / 16  # Ajuste o número de passos por segundo conforme necessário
                end = (note + 1) / 16
                midi_note = pretty_midi.Note(velocity=100, pitch=pitch + 21, start=start, end=end)
                instrument.notes.append(midi_note)

        pm.instruments.append(instrument)
        pm.write(midi_file)

    def midi_to_wav(self, midi_file, wav_file):
        subprocess.run(
            [
                self.fluidsynth_path,
                "-ni",
                self.soundfont_path,
                midi_file,
                "-F", wav_file,
                "-r", str(self.sample_rate)
            ],
            check=True
        )

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
      npy_file = self.file_paths[idx]
      piano_roll = np.load(npy_file)

      # Arquivos de saída
      midi_file = os.path.join(self.output_directory, f"{os.path.basename(npy_file).replace('.npy', '.mid')}")
      wav_file = os.path.join(self.output_directory, f"{os.path.basename(npy_file).replace('.npy', '.wav')}")

      # Converte piano roll para MIDI e MIDI para WAV
      self.piano_roll_to_midi(piano_roll, midi_file)
      self.midi_to_wav(midi_file, wav_file)

      # Carregar áudio como tensor
      waveform, _ = torchaudio.load(wav_file)

      # Converter para mono se necessário
      if waveform.size(0) == 2:  # Áudio estéreo
          waveform = waveform.mean(dim=0, keepdim=True)  # Converte para mono

      # Ajustar o tamanho do waveform
      if waveform.size(1) > self.num_samples:
          waveform = waveform[:, :self.num_samples]  # Trunca se maior
      elif waveform.size(1) < self.num_samples:
          padding = self.num_samples - waveform.size(1)  # Calcula o padding
          waveform = torch.nn.functional.pad(waveform, (0, padding), mode='constant', value=0)  # Preenche com zeros

      # Adicionar canal extra
      waveform = waveform.unsqueeze(0)  # (1, 1, num_samples)

      # Debug para validar a forma do tensor
      #print(f"Tamanho do waveform final: {waveform.shape}")

      return waveform

In [ ]:
from torch.utils.data import random_split

# Configurações
npy_directory = "/content/drive/MyDrive/diffusion/JCP_mixed"
output_directory = "/content/drive/MyDrive/diffusion/data_output"
soundfont_path = "/content/drive/MyDrive/diffusion/GeneralUser_GS_v2.0.1--doc_r3/GeneralUser-GS/GeneralUser-GS.sf2"
fluidsynth_path = "/usr/bin/fluidsynth"

# Cria o Dataset e DataLoader
dataset = NpyToWavDataset(npy_directory, output_directory, soundfont_path, fluidsynth_path)

# Tamanho total do dataset
dataset_size = len(dataset)

# Proporção de treino e validação
train_ratio = 0.1
val_ratio = 0.01

# Cálculo dos tamanhos
train_size = int(dataset_size * train_ratio)
val_size = int(dataset_size * val_ratio)
diff = dataset_size - train_size - val_size

# Divisão aleatória
train_dataset, val_dataset, diff_ = random_split(dataset, [train_size, val_size, diff])

# Criação dos DataLoaders
batch_size = 4
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=8)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=8)

print(f"Tamanho do conjunto de treino: {len(train_dataset)}")
print(f"Tamanho do conjunto de validação: {len(val_dataset)}")

# Itera pelo DataLoader
#for batch in train_dataloader:
#    print(batch.shape)  # Cada batch será um tensor com waveforms


Tamanho do conjunto de treino: 357
Tamanho do conjunto de validação: 35


In [ ]:
import torch
from audio_diffusion_pytorch import DiffusionModel, UNetV0, VDiffusion, VSampler

# Configuração do modelo
#model = DiffusionModel(
#    net_t=UNetV0,
#    in_channels=1,  # Áudio mono
#    channels=[8, 32, 64, 128, 256, 512, 512, 1024, 1024],
#    factors=[1, 4, 4, 4, 2, 2, 2, 2, 2],
#    items=[1, 2, 2, 2, 2, 2, 2, 4, 4],
#    attentions=[0, 0, 0, 0, 0, 1, 1, 1, 1],
#    attention_heads=8,
#    attention_features=64,
#    diffusion_t=VDiffusion,
#    sampler_t=VSampler,
#)  # Mova o modelo para a GPU, se disponível

# Arquitetura mais simples
model = DiffusionModel(
    net_t=UNetV0,
    in_channels=1,
    channels=[8, 16],  # Reduzindo para duas camadas
    factors=[1, 2],
    items=[1, 1],
    attentions=[0, 1],
    attention_heads=2,
    attention_features=8,
    diffusion_t=VDiffusion,
    sampler_t=VSampler,
)

#from torch import nn

#for name, module in model.named_modules():
#    if isinstance(module, nn.GroupNorm):
#        print(f"Ajustando GroupNorm no módulo: {name}")
#        num_channels = module.num_channels
#        new_layer = nn.LayerNorm([num_channels])  # Substitui por LayerNorm
#        setattr(module, name.split('.')[-1], new_layer)


# Otimizador
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [ ]:
#def align_tensors_hook(module, input, output):
#    # Ajusta o tamanho da dimensão temporal (dimensão 2)
#    if isinstance(output, torch.Tensor) and output.dim() == 3:
#        min_length = min(t.size(2) for t in input if isinstance(t, torch.Tensor) and t.dim() == 3)
#        if output.size(2) > min_length:
#            return output[:, :, :min_length]  # Trunca para o menor tamanho
#        elif output.size(2) < min_length:
#            padding = min_length - output.size(2)
#            return torch.nn.functional.pad(output, (0, padding))  # Adiciona padding
#    return output
#
## Registra o hook
#for name, module in model.named_modules():
#    if isinstance(module, nn.Conv1d):  # Foco em camadas convolucionais
#        module.register_forward_hook(align_tensors_hook)
#

In [ ]:
import torch
import os
import pandas as pd
import torch_xla.core.xla_model as xm

# Configurações
save_path = "/content/drive/MyDrive/diffusion/saved_models"  # Diretório para salvar os modelos no Google Drive
os.makedirs(save_path, exist_ok=True)

# Verificar GPU/TPU
device = xm.xla_device()  # Dispositivo XLA (TPU)
print(f"Usando dispositivo: {device}")

# Mover modelo para o dispositivo
model = model.to(device)

# Configurações para salvar os dados
train_losses = []  # Lista para armazenar as perdas do treinamento
val_losses = []    # Lista para armazenar as perdas da validação

# Função para calcular a perda no conjunto de validação
def validate(model, val_dataloader):
    model.eval()  # Coloca o modelo em modo de validação
    val_loss = 0.0
    with torch.no_grad():  # Desativa o cálculo de gradiente para validação
        for batch in val_dataloader:
            batch = batch.to(device)
            #print(f"Entrada do modelo val - Shape: {batch.shape}, Tipo: {type(batch)}, Device: {batch.device}")

            # Ajustar a forma do batch
            if batch.dim() == 4 and batch.size(2) == 1:
              batch = batch.squeeze(2)  # Remove a dimensão extra

            if batch.size(-1) != 16000:
              if batch.size(-1) > 16000:
                batch = batch[..., :16000]
              else:
                padding = 16000 - batch.size(-1)
                batch = torch.nn.functional.pad(batch, (0, padding))

            loss = model(batch)
            val_loss += loss.item()
    return val_loss / len(val_dataloader)  # Média das perdas

# Loop de treinamento
num_epochs = 30  # Defina o número de épocas conforme necessário
for epoch in range(num_epochs):
    #torch.cuda.empty_cache()
    model.train()  # Coloca o modelo em modo de treinamento
    epoch_train_loss = 0.0

    # Loop de treinamento
    total_batches = len(train_dataloader)
    for batch_idx, batch in enumerate(train_dataloader, start=1):
        batch = batch.to(device)
        #print(f"Entrada do modelo train - Shape: {batch.shape}, Tipo: {type(batch)}, Device: {batch.device}")

      # Ajustar a forma do batch
        if batch.dim() == 4 and batch.size(2) == 1:
          batch = batch.squeeze(2)  # Remove a dimensão extra

        if batch.size(-1) != 16000:
          if batch.size(-1) > 16000:
            batch = batch[..., :16000]
          else:
            padding = 16000 - batch.size(-1)
            batch = torch.nn.functional.pad(batch, (0, padding))


        print(f"[Época {epoch + 1}/{num_epochs}] Batch {batch_idx}/{total_batches} - Forma do batch ajustado: {batch.shape}")

        try:
            optimizer.zero_grad()
            loss = model(batch)  # Calcula a perda
            loss.backward()
            xm.optimizer_step(optimizer)  # Atualização de gradientes na TPU
            epoch_train_loss += loss.item()

            # Imprime a perda do batch
            print(f"  [Batch {batch_idx}] Loss: {loss.item():.4f} - Restam {total_batches - batch_idx} batches na época")

            # Salvar o modelo após cada época
            model_save_path = f"{save_path}/model_epoch_{epoch + 1}_{batch_idx}.pt"
            torch.save(model.state_dict(), model_save_path)
            #output = model(batch)
            #if output.dim() == 0:
              #print(f"Saída escalar no batch {batch_idx}. Output: {output.shape}")
              #print('--')
        except RuntimeError as e:
          print(f"Erro durante o treinamento no batch {batch_idx}: {e}")
          continue  # Ignora o batch problemático e continua

    # Cálculo da perda média no treinamento
    epoch_train_loss /= total_batches
    train_losses.append(epoch_train_loss)

    # Cálculo da perda no conjunto de validação
    epoch_val_loss = validate(model, val_dataloader)
    val_losses.append(epoch_val_loss)

    # Salvar o modelo após cada época
    model_save_path = f"{save_path}/model_epoch_{epoch + 1}.pt"
    torch.save(model.state_dict(), model_save_path)
    print(f"Modelo salvo em {model_save_path}")

    # Exibir resultados da época
    print(f"Época {epoch + 1}/{num_epochs}, Loss Treino: {epoch_train_loss:.4f}, Loss Validação: {epoch_val_loss:.4f}")

# Salvar as perdas em um arquivo
losses_df = pd.DataFrame({"Train Loss": train_losses, "Validation Loss": val_losses})
losses_df.to_csv(f"{save_path}/losses.csv", index=False)
print(f"Treinamento concluído. Dados de perda salvos em {save_path}/losses.csv.")

/usr/local/lib/python3.10/dist-packages/torch_xla/__init__.py:253: UserWarning: `tensorflow` can conflict with `torch-xla`. Prefer `tensorflow-cpu` when using PyTorch/XLA. To silence this warning, `pip uninstall -y tensorflow && pip install tensorflow-cpu`. If you are in a notebook environment such as Colab or Kaggle, restart your notebook runtime afterwards.
  warnings.warn(


Usando dispositivo: xla:0
[Época 1/30] Batch 1/90 - Forma do batch ajustado: torch.Size([4, 1, 16000])
  [Batch 1] Loss: 0.3686 - Restam 89 batches na época
[Época 1/30] Batch 2/90 - Forma do batch ajustado: torch.Size([4, 1, 16000])
  [Batch 2] Loss: 0.0342 - Restam 88 batches na época
[Época 1/30] Batch 3/90 - Forma do batch ajustado: torch.Size([4, 1, 16000])
  [Batch 3] Loss: 0.3876 - Restam 87 batches na época
[Época 1/30] Batch 4/90 - Forma do batch ajustado: torch.Size([4, 1, 16000])
  [Batch 4] Loss: 0.3142 - Restam 86 batches na época
[Época 1/30] Batch 5/90 - Forma do batch ajustado: torch.Size([4, 1, 16000])
  [Batch 5] Loss: 0.4025 - Restam 85 batches na época
[Época 1/30] Batch 6/90 - Forma do batch ajustado: torch.Size([4, 1, 16000])
  [Batch 6] Loss: 0.3852 - Restam 84 batches na época
[Época 1/30] Batch 7/90 - Forma do batch ajustado: torch.Size([4, 1, 16000])
  [Batch 7] Loss: 0.1882 - Restam 83 batches na época
[Época 1/30] Batch 8/90 - Forma do batch ajustado: torch.

KeyboardInterrupt: 

In [ ]:
torch.save(model.state_dict(), f"{save_path}/model_weights.pth")
print("Pesos do modelo salvos como 'model_weights.pth'.")

Pesos do modelo salvos como 'model_weights.pth'.


In [ ]:
save_path = "/content/drive/MyDrive/diffusion/saved_models"  # Diretório para salvar os modelos no Google Drive
model.load_state_dict(torch.load(f"{save_path}/model_epoch_{5}.pt", map_location=device))

<ipython-input-11-48d198278813>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f"{save_path}/model_epoch_{5}.pt", map_location=device))


<All keys matched successfully>

In [ ]:
noise = torch.randn(1, 1, 16000).to(device) # [batch_size, in_channels, length]
sample = model.sample(noise, num_steps=10) # Suggested num_steps 10-100

In [ ]:
if sample.dim() == 3:
    # Se o tensor tem 3 dimensões (batch, canal, comprimento), remova o batch
    generated_audio = sample.squeeze(0)  # Remove a dimensão do batch

In [ ]:
# Salvar o áudio gerado (se bem-sucedido)
if 'generated_audio' in locals():
    import torchaudio
    torchaudio.save(f"{save_path}/generated_audio.wav", generated_audio.cpu(), sample_rate=22050)
    print("Áudio gerado e salvo com sucesso!")

Áudio gerado e salvo com sucesso!


In [ ]:
def sample_audio(model, num_steps=50, initial_noise=None):
    """
    Função para gerar áudio amostrando do modelo de difusão.
    """
    device = next(model.parameters()).device  # Obtém o dispositivo do modelo
    model.eval()  # Coloca o modelo em modo de avaliação

    # Gera ruído inicial se não fornecido
    if initial_noise is None:
        initial_noise = torch.randn(1, 1, 16000).to(device)  # Certifique-se de que está no dispositivo correto

    # Move o ruído inicial explicitamente para o dispositivo do modelo
    initial_noise = initial_noise.to(device)

    # Iteração reversa para geração de áudio
    current_audio = initial_noise
    print(f"Forma do ruído inicial: {current_audio.shape}")

    for step in range(num_steps):
        # Use o método forward do modelo para ajustar o áudio
        with torch.no_grad():
            output = model.diffusion(current_audio)  # Corrigido para usar o método correto

            # Adicione verificação para saída válida
            if output.dim() == 0 or output.shape != current_audio.shape:
                raise ValueError(
                    f"Erro na geração no passo {step + 1}: saída inválida com forma {output.shape}"
                )

            current_audio = output  # Atualiza o áudio gerado
        print(f"Passo {step + 1}/{num_steps}: Forma do áudio gerado: {current_audio.shape}")

    return current_audio

In [ ]:
# Modificar temporariamente o método forward para debugging
def debug_forward(self, x, **kwargs):
    print(f"Forward chamado com entrada: {x.shape}")
    result = super(type(self), self).forward(x, **kwargs)
    if result.dim() == 0 or result.shape != x.shape:
        print(f"Forward saída inválida: {result.shape}")
    return result

# Substituir temporariamente o método forward para inspeção
model.diffusion.forward = debug_forward.__get__(model.diffusion, type(model.diffusion))

In [ ]:
from torch import nn

class VDiffusion(nn.Module):
    def forward(self, x, **kwargs):
        # Exemplo básico de difusão
        print(f"VDiffusion forward: Entrada {x.shape}")
        # Simular uma transformação básica
        return x * 0.9  # Exemplo: Escala de 90%

In [ ]:
#from torch.nn import Module

model.diffusion = VDiffusion()

In [ ]:
# Ruído inicial
initial_noise = torch.randn(1, 1, 16000).to(device)

# Geração de áudio
try:
    generated_audio = sample_audio(model, num_steps=500, initial_noise=initial_noise)
except ValueError as e:
    print(f"Erro durante a geração: {e}")

# Garantir que o tensor seja 2D para salvar corretamente (canal x comprimento)
if generated_audio.dim() == 3:
    # Se o tensor tem 3 dimensões (batch, canal, comprimento), remova o batch
    generated_audio = generated_audio.squeeze(0)  # Remove a dimensão do batch

# Verificar forma do tensor final
print(f"Forma final do áudio gerado: {generated_audio.shape}")

# Certificar que o tensor está no formato correto para salvar
if generated_audio.dim() != 2:
    raise ValueError("O áudio gerado não está no formato correto (canal x comprimento).")

# Salvar o áudio gerado (se bem-sucedido)
if 'generated_audio' in locals():
    import torchaudio
    torchaudio.save(f"{save_path}/generated_audio.wav", generated_audio.cpu(), sample_rate=22050)
    print("Áudio gerado e salvo com sucesso!")

Forma do ruído inicial: torch.Size([1, 1, 16000])
VDiffusion forward: Entrada torch.Size([1, 1, 16000])
Passo 1/500: Forma do áudio gerado: torch.Size([1, 1, 16000])
VDiffusion forward: Entrada torch.Size([1, 1, 16000])
Passo 2/500: Forma do áudio gerado: torch.Size([1, 1, 16000])
VDiffusion forward: Entrada torch.Size([1, 1, 16000])
Passo 3/500: Forma do áudio gerado: torch.Size([1, 1, 16000])
VDiffusion forward: Entrada torch.Size([1, 1, 16000])
Passo 4/500: Forma do áudio gerado: torch.Size([1, 1, 16000])
VDiffusion forward: Entrada torch.Size([1, 1, 16000])
Passo 5/500: Forma do áudio gerado: torch.Size([1, 1, 16000])
VDiffusion forward: Entrada torch.Size([1, 1, 16000])
Passo 6/500: Forma do áudio gerado: torch.Size([1, 1, 16000])
VDiffusion forward: Entrada torch.Size([1, 1, 16000])
Passo 7/500: Forma do áudio gerado: torch.Size([1, 1, 16000])
VDiffusion forward: Entrada torch.Size([1, 1, 16000])
Passo 8/500: Forma do áudio gerado: torch.Size([1, 1, 16000])
VDiffusion forward: En

In [ ]:
output_path = f"{save_path}/generated_audio_raw.wav"
torchaudio.save(output_path, generated_waveform.cpu(), sample_rate)
print(f"Áudio gerado e salvo em: {output_path}")

#generated_waveform = generated_waveform.squeeze(0).cpu()  # Remove batch dimension
generated_waveform = generated_waveform / generated_waveform.abs().max()  # Normaliza entre -1 e 1
output_path = f"{save_path}/generated_audio_norm.wav"
torchaudio.save(output_path, generated_waveform.cpu(), sample_rate)
print(f"Áudio gerado e salvo em: {output_path}")

Áudio gerado e salvo em: /content/drive/MyDrive/diffusion/saved_models/generated_audio_raw.wav
Áudio gerado e salvo em: /content/drive/MyDrive/diffusion/saved_models/generated_audio_norm.wav
